In [3]:
from lib import db_config

In [4]:
import pandas as pd
import string

In [5]:
school_code = '173428'

In [27]:
q = "select  a.school_code , a.class, a.`test_edition`, group_concat(distinct a.pan_number order by a.pan_number) as pan\
    from assessment{} as a\
    WHERE a.school_code = %s and a.`pan_number` <> 0\
    group by a.`school_code`, a.class"%school_code

In [28]:
q

'select  a.school_code , a.class, a.`test_edition`, group_concat(distinct a.pan_number order by a.pan_number) as pan    from assessment{} as a    WHERE a.school_code = 173428 and a.`pan_number` <> 0    group by a.`school_code`, a.class'

In [29]:
l = list(string.ascii_uppercase)
n = list(string.digits)
l.append('37')

l = l+n
l.remove('0')

Read all the data from the tables

In [33]:
df_school = pd.DataFrame()
for i in l:
    query = q.format(i)
    try : 
        df_temp = db_config.connect('educat', query)
        df_school = df_school.append(df_temp, ignore_index=True)
    except:
          print("Couldn't read : %s"%i)
# data.to_pickle('data/asset_cohorts.pkl')

<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>


/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 103 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 206 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 309 was cut by GROUP_CONCAT()')
  result = self._query(query)


<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
Couldn't read : X
<<CONNECTED>>
Couldn't read : Y
<<CONNECTED>>
<<CONNECTED>>


/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 412 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 515 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 618 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 721 was cut by GROUP_CONCAT()')
  result = self._query(query)


<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>
<<CONNECTED>>


In [35]:
# school_name = df_school.school_name.value_counts().index[0]
# school_board = df_school.primary_board.value_counts().index[0]
# school_city = df_school.city.value_counts().index[0]

In [36]:
df_school.to_pickle('data/%s - asset_cohorts.pkl'%school_code)

Getting all the data in one DF

In [9]:
data = pd.read_pickle('data/asset_cohorts.pkl')

# data = pd.DataFrame()

# l = list(string.ascii_uppercase)
# l.append('37')

# for i in l:
#     query = q.format(i)
#     df_temp = db_config.connect('educat', query)
#     data = data.append(df_temp, ignore_index=True)
# data.to_pickle('data/asset_cohorts.pkl')

In [37]:
data = df_school
data.sort_values('pan', inplace=True)

Convert to set

In [38]:
to_set = lambda x : set(map(int, set(x.split(','))))

data['pan'] = data['pan'].apply(to_set) 

In [45]:
for index, row in data.iterrows():
    pan = row['pan']
    print(index)
    temp = data[~index]


8


KeyError: -9

Looking at the sets of students in the current year and creating cohorots that can be looked up with the past data

In [40]:
data_Z = db_config.connect('educat', q.format('37'));

data_Z2 = db_config.connect('educat', q.format('Z'));

data_latest = data_Z.append(data_Z2, ignore_index=True)

data_latest['pan'] = data_latest.pan.apply(to_set)

data_latest['len'] = data_latest.pan.apply(len)

cohorts = data_latest[data_latest.len > 10].pan

<<CONNECTED>>


/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 103 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 206 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 309 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 412 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 515 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1260, 'Row 618 was cut by GROUP_CONCAT()')
  result = self._query(query)
/Users/manghat/anaconda3/lib/python3.6/s

<<CONNECTED>>


Contains the list of cohorts that are there in school and class combinations that are in the same cohorts

In [ ]:
cohorts

In [23]:
chrt = pd.DataFrame()

In [25]:
for cohort in cohorts:
    diff = lambda x :  len(x & cohort)  / len(x | cohort) #+ (x + cohort)
    temp = data
    temp['diff'] = temp.pan.apply(diff)
    temp = temp[temp['diff'] > 0.5]
    temp.sort_values('diff', inplace=True, ascending=True)
    if len(temp):   
        chrt.append(temp)
        chrt['ref'] = str(cohort)

/Users/manghat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/manghat/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [26]:
temp.sort_values('diff', ascending=True)

,school_code,class,test_edition,pan,diff


In [ ]:
! pip install pandas-sets